<a href="https://colab.research.google.com/github/chetools/CHE4061_Spring2026/blob/main/NRTL_VLEFlash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
%run che5.ipynb

In [2]:
p=Props(['Isopropanol','Water','Toluene'])

In [3]:
p.NRTL_gamma(np.array([0.3,0.3,0.4]), 300.)

Array([1.12930337, 6.26607585, 3.13419067], dtype=float64)

In [21]:
#NRTL Parameters: Tij = Aij + Bij/T + Cij * Ln(T) + Dij * T
def NRTL_Gex(x, T):
    tau = p.NRTL_A+p.NRTL_B/T + p.NRTL_C*jnp.log(T)+p.NRTL_D*T
    G = jnp.exp(-p.NRTL_alpha*tau)
    tauG = tau*G
    tauGx = jnp.einsum('ji,j->i', tauG, x)
    Gx = jnp.einsum('ki,k->i', G, x)
    return jnp.sum(x*tauGx/Gx)

In [24]:
Gex_grad = jax.grad(NRTL_Gex, 0)

In [26]:
jnp.exp(Gex_grad(x,T))

Array([1.12930337, 6.26607585, 3.13419067], dtype=float64)

In [22]:
x=np.array([0.3,0.3,0.4])
T=300.
NRTL_Gex(x,T)

Array(1.04397377, dtype=float64)

In [60]:
def NRTL(x, T):
    tau = p.NRTL_A+p.NRTL_B/T + p.NRTL_C*jnp.log(T)+p.NRTL_D*T
    G = jnp.exp(-p.NRTL_alpha*tau)
    tauG = tau*G
    xtauG = jnp.einsum('ji,j->i', tauG, x)
    xG = jnp.einsum('ki,k->i', G, x)
    xtauGdivxG = xtauG/xG
    return jnp.exp(xtauGdivxG + jnp.einsum('j,ij->i',x,G*(tau - xtauGdivxG[None,:])/xG))


In [61]:
NRTL(x,T)

Array([1.12930337, 6.26607585, 3.13419067], dtype=float64)

In [64]:
def bubbleP(x, T):
    yP = x*p.Pvap(T)
    P = np.sum(yP)
    return P, yP/P

In [65]:
def bubbleP_NRTL(x,T):
    yP = x*p.NRTL_gamma(x,T)* p.Pvap(T)
    P = np.sum(yP)
    return P, yP/P

In [66]:
bubbleP(x,T)

(Array(4775.09032435, dtype=float64),
 Array([0.42660746, 0.22317803, 0.35021451], dtype=float64))

In [67]:
bubbleP_NRTL(x,T)

(Array(14219.54443835, dtype=float64),
 Array([0.16178378, 0.46961611, 0.36860011], dtype=float64))

In [63]:
def dewP(y, T):
    Pvap = p.Pvap(T)
    P = 1/np.sum(y/Pvap)
    return P, y*P/Pvap

In [ ]:
def bubbleT(x, P):

    T0 = np.sum(x*p.Tb(P))
    def froot(T):
        return bubbleP(x,T)[0]-P

    T=sp.optimize.root_scalar(froot, x0=T0, method='secant').root
    return T, x*p.Pvap(T)/P

In [ ]:
def dewT(y, P):

    T0 = np.sum(y*p.Tb(P))
    def froot(T):
        return dewP(y,T)[0]-P

    T=sp.optimize.root_scalar(froot, x0=T0, method='secant').root
    return T, y*P/p.Pvap(T)

In [ ]:
#Const T, binary-ideal VLE
T= 360.
x1s = np.linspace(0,1,21)
Ps=[]
y1s=[]
for x1 in x1s:
    P, y = bubbleP(np.array([x1, 1-x1]), T)
    Ps.append(P)
    y1s.append(y[0])

fig = make_subplots()
fig.add_scatter(x=x1s, y=Ps, name='bubbleP')
fig.add_scatter(x=y1s, y=Ps, name='dewP')
fig.update_layout(width=600, height=400)

In [ ]:
#Const P, binary-ideal VLE
P= 1e5
x1s = np.linspace(0,1,21)
Ts=[]
y1s=[]
for x1 in x1s:
    T, y = bubbleT(np.array([x1, 1-x1]), P)
    Ts.append(T)
    y1s.append(y[0])

fig = make_subplots()
fig.add_scatter(x=x1s, y=Ts, name='bubbleT')
fig.add_scatter(x=y1s, y=Ts, name='dewT')
fig.update_layout(width=600, height=400)

In [ ]:
fig = make_subplots()
fig.add_scatter(x=x1s, y=y1s, mode='markers')
fig.add_scatter(x=[0.,1.],y=[0.,1.],mode='lines', line_color='rgba(100,100,100,0.5)')
alpha= 2.49
fig.add_scatter(x=x1s, y= alpha*x1s/(1-x1s+alpha*x1s), mode='lines', name='const alpha')
fig.update_layout(width=400, height=400, showlegend=False)

In [ ]:
y1=np.array(y1s[1:-1])
x1=np.array(x1s[1:-1])
np.average(y1*(1-x1)/(1.-y1)/(x1))

In [ ]:
def flashPT(z, P, T):
    dP, _ = dewP(z, T)
    bP, _ = bubbleP(z, T)
    if P>bP:
        return 'All Liquid'
    if P<dP:
        return 'All Gas'

    phi_guess = (bP - P)/(bP-dP)

    K=p.Pvap(T)/P

    def froot(phi):
        return jnp.sum(z*(K-1)/(1+phi*(K-1)))

    froot_prime= jax.grad(froot)

    froot = jax.jit(froot)

    phi=sp.optimize.root_scalar(froot, x0=phi_guess, method='Newton', fprime = froot_prime).root
    x = z/(1+phi*(K-1))
    y = K*x
    return phi, x, y

In [ ]:
flashPT(np.array([0.4,0.6]), 1e5, 370.)

In [ ]:
hs = np.logspace(-1,-15, 15)
x=np.pi/3
for h in hs:
    approx= (np.sin(x+h)-np.sin(x))/h
    exact = np.cos(x)
    print(h, exact, approx, np.abs((exact-approx)/exact))